# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
!conda install -y tqdm

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.5
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [2]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from tqdm import tqdm_notebook
import numpy as np
from PIL import Image
import os
import random
import math
from workspace_utils import keep_awake, active_session

In [ ]:
def cross_platform_keep_alive(iterable):
    import subprocess
    
    cp = subprocess.run(['uname', '-a'], capture_output=True)
    
    running_on_gcp = b'-gcp' in cp.stdout
    if running_on_gcp:
        print('running on gcp - use keep awake')
        return keep_awake(iterable)
    else:
        print('not running on gcp - dont use keep awake')
        return iterable

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [3]:
EPOCHS = 25
CLASS_COUNT = 102
CLASSIFIER_WIDTH = 512 #  specific to resnet18
CLASSIFIER_HIDDEN_WIDTH = 256
CLASSIFIER_DROP_PROBABILITY=0.6
LEARN_RATE = 0.001
DATA_AUGMENT_RANDOM_ROTATION = 30
NETWORK_INPUT_SIZE = 224
VALID_RESIZE_TRANSFORM = TEST_RESIZE_TRANSFORM = 255
BATCH_SIZE = 16
DATA_NORMALIZE_MEAN = [0.485, 0.456, 0.406]
DATA_NORMALIZE_STD = [0.229, 0.224, 0.225]

In [4]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz

--2019-09-26 11:57:40--  https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.9.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.9.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344873452 (329M) [application/x-gzip]
Saving to: ‘flower_data.tar.gz’

flower_data.tar.gz  100%[===================>] 328.90M  19.9MB/s    in 17s     

2019-09-26 11:57:57 (19.6 MB/s) - ‘flower_data.tar.gz’ saved [344873452/344873452]



In [5]:
!mkdir flowers
!tar -xf flower_data.tar.gz -C flowers
!rm flower_data.tar.gz

mkdir: cannot create directory ‘flowers’: File exists


In [6]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [7]:
# Define your transforms for the training, validation, and testing sets
data_transforms = {
    'training': transforms.Compose([
        transforms.RandomRotation(DATA_AUGMENT_RANDOM_ROTATION),
        transforms.RandomResizedCrop(NETWORK_INPUT_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            DATA_NORMALIZE_MEAN,
            DATA_NORMALIZE_STD
        )
    ]),
    'validation': transforms.Compose([
        transforms.Resize(VALID_RESIZE_TRANSFORM),
        transforms.CenterCrop(NETWORK_INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            DATA_NORMALIZE_MEAN,
            DATA_NORMALIZE_STD
        )
    ]),
    'testing': transforms.Compose([
        transforms.Resize(TEST_RESIZE_TRANSFORM),
        transforms.CenterCrop(NETWORK_INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            DATA_NORMALIZE_MEAN,
            DATA_NORMALIZE_STD
        )
    ])
}

# Load the datasets with ImageFolder
image_datasets = {
    'training': datasets.ImageFolder(train_dir, transform=data_transforms['training']),
    'validation': datasets.ImageFolder(valid_dir, transform=data_transforms['validation']),
    'testing': datasets.ImageFolder(test_dir, transform=data_transforms['testing'])
}

# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'training': torch.utils.data.DataLoader(image_datasets['training'], batch_size=BATCH_SIZE, shuffle=True),
    'validation': torch.utils.data.DataLoader(image_datasets['validation'], batch_size=BATCH_SIZE),
    'testing': torch.utils.data.DataLoader(image_datasets['testing'], batch_size=BATCH_SIZE)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [8]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [9]:
def replace_resnet18_classifier_layer(
    orig_model,
    classifier_width,
    hidden_width,
    dropout_prob,
    class_count
):
    orig_model.fc = nn.Sequential(
        nn.Linear(classifier_width, hidden_width),
        nn.ReLU(),
        nn.Dropout(p=dropout_prob),
        nn.Linear(hidden_width, class_count),
        nn.LogSoftmax(dim=1)
    )
    return orig_model

In [10]:
def device_detect():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# Build and train your network
device = device_detect()

def training_step(model, images, labels, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    logps = model.forward(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def eval_step(model, images, labels, criterion):
    images, labels = images.to(device), labels.to(device)
    logps = model.forward(images)
    step_loss += criterion(logps, labels).item()
    ps = torch.exp(logps)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    step_correct += torch.sum(equals.type(torch.FloatTensor)).item()
    return step_loss, step_correct

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model = replace_resnet18_classifier_layer(
    model,
    CLASSIFIER_WIDTH,
    CLASSIFIER_HIDDEN_WIDTH,
    CLASSIFIER_DROP_PROBABILITY,
    CLASS_COUNT
)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARN_RATE)

model = model.to(device)

for e in range(1, 1 + EPOCHS):
    print('start epoch {}'.format(e))
    model.train()
    training_loss = 0
    print('training')
    for images, labels in tqdm_notebook(cross_platform_keep_alive(dataloaders['training'])):
        training_loss += training_step(model, images, labels, optimizer, criterion)
    print('training_loss = {}'.format(training_loss / len(dataloaders['training'])))
        
    model.eval()
    validation_loss = 0
    correct_count = 0
    example_count = 0
    print('validation')
    with torch.no_grad():
        for images, labels in tqdm_notebook(cross_platform_keep_alive(dataloaders['validation'])):
            step_loss, step_correct = eval_step(model, images, labels, criterion)
            example_count += labels.shape[0]
            validation_loss += step_loss
            correct_count += step_correct
    print('validation_loss = {}, accuracy = {}'.format(
        validation_loss / len(dataloaders['validation']),
        correct_count / example_count
    ))

start epoch 1
training


KeyboardInterrupt: 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# Do validation on the test set
device = device_detect()
model.eval()
model = model.to(device)
testing_loss = 0
correct_count = 0
example_count = 0
print('testing')
with torch.no_grad():
    for images, labels in tqdm_notebook(cross_platform_keep_alive(dataloaders['testing'])):
        step_loss, step_correct = eval_step(model, images, labels, criterion)
        testing_loss += step_loss
        correct_count += step_Correct
        example_count += labels.shape[0]
print('testing_loss = {}, accuracy = {}'.format(
    testing_loss / len(dataloaders['testing']),
    correct_count / example_count
))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save the checkpoint 
model.class_to_idx = image_datasets['training'].class_to_idx
checkpoint = {
    'class_to_idx': image_datasets['training'].class_to_idx,
    'model_state_dict': model.state_dict(),    
    # Below entries are needed if training is being resumed on the checkpoint
    'optimizer_state_dict': optimizer.state_dict,
    'CLASSIFIER_HIDDEN_WIDTH': CLASSIFIER_HIDDEN_WIDTH,
    'CLASSIFIER_DROP_PROBABILITY': CLASSIFIER_DROP_PROBABILITY,
}
torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    m = models.resnet18() # NOT pretrained
    m = replace_resnet18_classifier_layer(
        m,
        CLASSIFIER_WIDTH,
        checkpoint['CLASSIFIER_HIDDEN_WIDTH'],
        checkpoint['CLASSIFIER_DROP_PROBABILITY'],
        CLASS_COUNT
    )
    m.load_state_dict(checkpoint['model_state_dict'])
    m.class_to_idx = checkpoint['class_to_idx']
    return m

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def image_resize(image, min_dim_size=256):
    curwidth, curheight = image.size
    
    # resizeing is based on the smaller dimension
    if curheight < curwidth:
        newheight = min_dim_size
        newwidth = int(math.ceil(curwidth * newheight / curheight))
    else:
        newwidth = min_dim_size
        newheight = int(math.ceil(curheight * newwidth / curwidth))
    
    # in-place resize
    image.thumbnail((newwidth, newheight))
    
    #sanity checks
    assert image.size[0] >= min_dim_size
    assert image.size[1] >= min_dim_size
    
def image_square_center_crop(image, size=224):
    '''
    |---------------|   ^
    |      ^        |   |
    |      |6       |   |
    | 5 |-------| 5 |   |
    |<->| 3  |  |<->|   |
    |   |<---|->|   |   | 2
    |   |    |4 |   |   |
    |   |-------|   |   |
    |      ^        |   |
    |      |6       |   |
    |---------------|   |
    <--------------->
            1
    1: image width(image.size[0])
    2: image height(image.size[1])
    3=4 : size(square crop)
    5: width margin
    6: height margin
    '''
    width_margin = image.size[0] - size
    height_margin = image.size[1] - size
    left = int(0.5 * width_margin)
    top = int(0.5 * height_margin)
    right = left + size
    bottom = top + size
    image = image.crop((left, top, right, bottom))
    return image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Process a PIL image for use in a PyTorch model
    image_resize(image)
    image = image_square_center_crop(image)
    
    # pilow to numpy
    imarray = np.array(image)
    imarray = imarray.astype(np.float64)
    imarray /= 255

    # mean and standard deviation adjustments
    imarray -= np.array([0.485, 0.456, 0.406])
    imarray /= np.array([0.229, 0.224, 0.225])

    # pilow axis to pytorch axis
    imarray = imarray.transpose((2, 0, 1))
    
    return torch.tensor(imarray, dtype=torch.double)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
img = Image.open('flowers/train/1/image_06734.jpg')
procimg = process_image(img)
imshow(procimg)
fig, ax = plt.subplots()
ax.imshow(img)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # Implement the code to predict the class from an image file
    device = device_detect()
    
    # configure the model
    model.eval()
    model.double()
    
    img = process_image(Image.open(image_path))
    
    # device move
    model, img = model.to(device), img.to(device)

    # present a view that is like a batch of 1
    img = img.view(1, img.shape[0], img.shape[1], img.shape[2])
    with torch.no_grad():
        logps = model.forward(img)
        ps = torch.exp(logps)
        top_ps, top_classes = ps.topk(topk)
        
    # batch of size 1
    assert top_ps.shape[0] == 1
    top_ps = top_ps[0]
    assert top_classes.shape[0] == 1
    top_classes = top_classes[0]
    
    # invert the class_to_idx dictionary
    idx_to_class = {
        v: k for (k, v) in model.class_to_idx.items()
    }
    
    # translatate index to class numbers
    classes = [
        idx_to_class[int(x)] for x in top_classes
    ]
    
    # from float tensor back to native list of floats
    probs = [
        float(x) for x in top_ps
    ]
    return probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Display an image along with the top 5 classes

def pick_random_test_image(test_dir):
    test_dir_subdirectories = [
        x for x in os.listdir(
            test_dir
        ) if os.path.isdir(
            os.path.join(test_dir, x)
        )
    ]
    ground_truth_cat_num = random.choice(test_dir_subdirectories)
    ground_truth_cat_name = cat_to_name[ground_truth_cat_num]
    random_subdirectory = os.path.join(test_dir, ground_truth_cat_num)
    subdir_files = [
        x for x in os.listdir(
            random_subdirectory
        ) if os.path.isfile(
            os.path.join(random_subdirectory, x)
        ) and x[-4:].lower() == '.jpg'
    ]
    random_file_path = os.path.join(random_subdirectory, random.choice(subdir_files))
    return random_file_path, ground_truth_cat_name


def plot_image_and_prediction(image_path, probs, cat_names):
    fig, (ax_img, ax_prob) = plt.subplots(nrows=2)
    img = process_image(Image.open(image_path))
    imshow(img, ax=ax_img)
    
    # adapted from https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barh.html
    y_pos = np.arange(len(cat_names))
    ax_prob.barh(y_pos, probs, align='center')
    ax_prob.set_yticks(y_pos)
    ax_prob.set_yticklabels(cat_names)
    ax_prob.invert_yaxis()
    ax_img.set_title(ground_truth_label)
    plt.show()


test_file, ground_truth_label = pick_random_test_image(test_dir)
print(test_file)
probs, cats = predict(test_file, model)
cat_names = [cat_to_name[x] for x in cats]
plot_image_and_prediction(test_file, probs, cat_names)